# Anisotropic diffusion equation in 1D example

In [2]:
using LinearAlgebra
using Plots

using Pkg
Pkg.activate()
using SBP_operators

Going to solve the diffusion equation,
$$
    \frac{\partial u}{\partial t} = \frac{\partial}{\partial x}\left(k \frac{\partial u}{\partial x}\right),
$$
with boundary conditions,
$$
    u(0,t) = \cos(t),  \qquad u(L,t) = 0
$$
and initial condition,
$$
    u(x,0) = 0.
$$
This has corresponding exact solution
$$
    u(x,t) = 
$$

## Appendix: Solution for PDE

The homogeneous equation has eigenfunctions

$$
    X(x) = \sin(\lambda^2 x), \qquad \lambda^2 = \frac{n\pi}{L}
$$


Hence, setting $u(x,t) = v(x,t) + w(x,t)$, letting $w(x,t) = \frac{x-L}{L}\cos(t)$ to eliminate the boundary conditions and assuming $v(x,t) = \sum_N T_n(t) \sin(\lambda^2 x)$ we get, 
$$
\begin{align}
    \sum_N \frac{\partial T_n}{\partial t}\sin(\lambda^2 x) + k T_n(t) [\lambda^4 \sin(\lambda^2 x)] &= -\frac{x-L}{L}\sin(t),
\end{align}
$$
with boundary and initial conditions,
$$
\begin{align}
    v(0,t) &= u(0,t) - \cos(t) = 0   \\
    v(0,t) &= u(L,t) = 0    \\
    u(x,0) = 0 &= v(x,0) + w(x,0) \implies v(x,0) = -\frac{x-L}{L} 
\end{align}
$$



This is now a system of DEs which can be solve by integrating factor

In [5]:
mutable struct solution
    x   :: Vector{Float64}  # Grid
    u   :: Matrix{Float64}  # Solution
    Δx  :: Float64          # Grid size
    n   :: Int64            # Size of grid
    Δt  :: Float64          # Step size

    function solution(domain,initial_condition,Δt,Δx;t_final=10)
        x = collect(range(domain[1],domain[2],step=Δx))

        #Julia uses column-wise storage
        u = zeros(Float64,length(x),ceil(Int64,t_final/Δt))

        if typeof(initial_condition) <: Function
            u[:,1] = initial_condition(x)
        else
            u[:,1] = initial_condition
        end
        
        time_length = Int64(t_final/Δt) #Use t_final as a guess if unknown how long it will take, this helps with pre-allocation

        new(x,u,Δx,length(x),Δt)
    end
end

The RHS of the PDE with the included boundary term is $k\partial_{xx} u + SAT$. In this case the $SAT=\tau(u_0 - cos(t))$.

In [ ]:
function ∂ₓₓu(u,x,t,g,Δx,Δt,k;order=2)

    if order == 2
        τ₀ = 1.0/(0.5*Δx)
    elseif order == 4
        τ₁ = 0.5
    end

    uₓₓ = -k*SBP_operators.Dₓₓ!(uₓₓ,u,n,Δx,order=order)
    
    # SAT term
    u[1] = u[1] - τ(u[1] - g(t))
end

Set the initial parameters for the simulation

In [7]:
# Spatial domain
𝒟 = [0,2π]
n =  100
Δx = 𝒟[2]/n

# Time domain
Δt = 0.1
t_f = 10 #Length of the simulation

u₀(x) = -cos.(x) #Set initial condition

soln = solution(domain,u₀,Δt,Δx) #Build solution struct


order = 2 #Order of the integrator
method = :euler #Time stepping method


2

In [ ]:
function time_solver(soln,RHS,Δt,Δx,k,t_f,boundary;method=:euler)

    N = ceil(Int64,t_f/Δt)
    # n = size(u)[1]
    n = soln.n
    Δx = soln.Δx

    u = soln.u

    
    # Eulers method
    for i = 1:N-1
        t = i*soln.Δt
        if method == :euler
            soln.u[:,i+1] = soln.u[:,i] + Δt*RHS(soln.u[:,i+1],soln.u[:,i],n,Δx,Δt,k,t,x,boundary)

        elseif method == :rk4
            k1 = RHS(u[:,i+1],u[:,i]        ,n,Δx,Δt,k,t        ,x,boundary)
            k2 = RHS(u[:,i+1],u[:,i]+Δt/2*k1,n,Δx,Δt,k,t+0.5Δt  ,x,boundary)
            k3 = RHS(u[:,i+1],u[:,i]+Δt/2*k2,n,Δx,Δt,k,t+0.5Δt  ,x,boundary)
            k4 = RHS(u[:,i+1],u[:,i]+Δt*k3  ,n,Δx,Δt,k,t+Δt     ,x,boundary)

            u[:,i+1] = u[:,i] + Δt/6 * (k1 + 2k2 + 2k3 + k4)

        end
    end

    return soln
end